In [9]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Configuración general
np.random.seed(42)  # Para reproducibilidad
n_days = 180  # Número de días (180 días ≈ 6 meses)

In [16]:
# Parámetros de las plataformas
platforms = {
    'Google Ads': {'cpm': 5, 'ctr': 0.02, 'alpha': 0.001, 'beta': 0.03},
    'Facebook Ads': {'cpm': 4, 'ctr': 0.015, 'alpha': 0.0008, 'beta': 0.025},
    'Programmatic': {'cpm': 6, 'ctr': 0.012, 'alpha': 0.0006, 'beta': 0.02}
}

In [4]:
data = []

In [5]:
date = pd.Timestamp('2023-01-01')

In [6]:
for platform, params in platforms.items():
    print(platform)
    print(params)

Google Ads
{'cpm': 5, 'ctr': 0.02, 'alpha': 0.001, 'beta': 0.03}
Facebook Ads
{'cpm': 4, 'ctr': 0.015, 'alpha': 0.0008, 'beta': 0.025}
Programmatic
{'cpm': 6, 'ctr': 0.012, 'alpha': 0.0006, 'beta': 0.02}


In [7]:
# Parámetros para la distribución normal truncada del gasto
mean_spend = 600  # Media del gasto
std_spend = 150   # Desviación estándar del gasto

In [17]:
def generate_data(platform, params, n_days):
    data = []
    spend_values = np.random.normal(loc=mean_spend, scale=std_spend, size=n_days)
    
    for day in range(n_days):
        date = pd.Timestamp('2023-01-01') + pd.Timedelta(days=day)
        spend = spend_values[day]
        impressions = int((spend / params['cpm']) * 1000)
        clicks = int(impressions * params['ctr'])
        if impressions > 0:
            # Ajustar la tasa de conversión para que decrezca
            conversion_rate = params['alpha'] * np.log(impressions + 1) + params['beta']
            conversion_rate = max(0, min(conversion_rate, 1))  # Limitar entre 0 y 1
        else:
            conversion_rate = 0
        conversions = int(clicks * conversion_rate)
        data.append([date, platform, spend, impressions, clicks, conversions])
    
    return data

# Regenerar los datos ajustados para las tres plataformas
adjusted_data = []
for platform, params in platforms.items():
    platform_data = generate_data(platform, params, n_days)
    adjusted_data.extend(platform_data)

# Convertir a DataFrame con los datos ajustados
columns = ['Fecha', 'Plataforma', 'Gasto (€)', 'Impresiones', 'Clics', 'Conversiones']
df = pd.DataFrame(adjusted_data, columns=columns)
df = df[df['Gasto (€)'] > 0]
# Añadir columnas de CTR y CPA al dataframe ajustado
df['CTR'] = df['Clics'] / df['Impresiones']
df['CPA'] = df['Gasto (€)'] / df['Conversiones']
df['CR'] = df['Clics'] / df['Conversiones']
df['CPA'].replace([float('inf'), -float('inf')], np.nan, inplace=True)  # Evitar valores infinitos

# Guardar el nuevo dataset
df.to_csv('data.csv', index=False)